In [202]:
import pandas as pd
import missingno as msno
from datetime import datetime
import numpy as np
import sklearn.compose._column_transformer
import seaborn as sns
import matplotlib.pyplot as plt

In [203]:
customers = pd.read_csv("../data/olist_customers_dataset.csv") # Client
geolocalisation = pd.read_csv("../data/olist_geolocation_dataset.csv") # Loc

order_items = pd.read_csv("../data/olist_order_items_dataset.csv") # Commandes items
order_payments = pd.read_csv("../data/olist_order_payments_dataset.csv") # Commandes paiement
order_reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv") # Commandes reviews
orders = pd.read_csv("../data/olist_orders_dataset.csv") # Commandes

sellers = pd.read_csv("../data/olist_sellers_dataset.csv") # Vendeurs
products = pd.read_csv("../data/olist_products_dataset.csv") # Produits
cat_trans = pd.read_csv('../data/product_category_name_translation.csv') # Translate


In [204]:
order_cust = pd.merge(left=customers, right=orders, how="left", left_on="customer_id", right_on="customer_id")
forder_cust = pd.merge(left=order_cust, right=order_items, how="outer", left_on="order_id", right_on="order_id")

forder_cust['total'] = forder_cust[["price", "freight_value"]].sum(axis=1)

## Recence 

In [205]:
dates = ['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date','order_estimated_delivery_date','shipping_limit_date']
forder_cust[dates] =forder_cust[dates].apply(pd.to_datetime, format='%Y-%m-%d')
date_1 = forder_cust.sort_values(by ='order_approved_at',ascending = False)


recence = date_1.groupby('customer_unique_id').agg(last_date =('order_approved_at','max')).reset_index()


recence['periode']=(date_1.order_approved_at.max()-recence.last_date).dt.days
recence.dropna(inplace=True)
recence.periode = recence.periode.astype(int)

def recence_transform(row):
    if row['periode'] <= 140:
        return 1
    if row['periode']> 140 and row['periode'] <= 280:
        return 2
    if row['periode']>280 and row['periode'] <= 420:
        return 3
    if row['periode']> 420 and row['periode'] <= 560:
        return 4
    if row['periode']> 560:
        return 5 

recence['r_value']=recence.apply(lambda row: recence_transform(row),axis=1)
recence.head()

,customer_unique_id,last_date,periode,r_value
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 11:11:18,116,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 18:25:44,118,1
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,541,4
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:49:17,325,3
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 20:06:52,292,3


In [206]:

liste1 =cat_trans.product_category_name.tolist()
liste2 = cat_trans.product_category_name_english.tolist()
dict = {}
for key in liste1:
    for value in liste2:
        dict[key] = value
        liste2.remove(value)
        break
products.product_category_name=products.product_category_name.map(dict)
order_reviews.drop(['review_comment_title','review_comment_message'],inplace=True,axis=1)
order_reviews['review_score'].dropna(inplace=True)

## Merging pour l'analyse des commandes

### reajustement du df payments pour obtenir une ligne = une commande

In [207]:
def get_mode(x):
    return x.mode().iloc[0]

In [208]:
p1 = order_payments.groupby('order_id').agg(payment_installment = ('payment_installments',get_mode),payment_type = ('payment_type',get_mode),total_pay =('payment_value','sum')).reset_index()


### Merging products and order_items et les regrouper par commandes uniques

In [209]:
order_items.drop(['seller_id','price','freight_value'],inplace=True,axis=1)
products_item = pd.merge(order_items,products,how ='left',left_on='product_id',right_on='product_id')
a = products_item.groupby(['order_id','product_category_name']).agg(total_commande=('order_item_id','count'),prod =('product_id','nunique')).reset_index()
c = a.groupby('order_id').agg(category=('product_category_name',get_mode),items_commande = ('total_commande','sum'),prod_commande=('prod','sum')).reset_index()
df_commande = pd.merge(p1,c,how='left',left_on='order_id',right_on='order_id')
df_commande
#Dataframe montrant par commande , le type de paiement le plus utilisé, le total de la commande en R$, la catégorie du produit le plus commandé, le nb d'items commandé et le nombre de produits différents commandés


,order_id,payment_installment,payment_type,total_pay,category,items_commande,prod_commande
0,00010242fe8c5a6d1ba2dd792cb16214,2,credit_card,72.19,cool_stuff,1.0,1.0
1,00018f77f2f0320c557190d7a144bdd3,3,credit_card,259.83,pet_shop,1.0,1.0
2,000229ec398224ef6ca0657da4fc703e,5,credit_card,216.87,furniture_decor,1.0,1.0
3,00024acbcdf0a6daa1e931b038114c75,2,credit_card,25.78,perfumery,1.0,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,3,credit_card,218.04,garden_tools,1.0,1.0
...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,1,boleto,343.40,housewares,1.0,1.0
99436,fffcd46ef2263f404302a634eb57f7eb,1,boleto,386.53,computers_accessories,1.0,1.0
99437,fffce4705a9662cd70adb13d4a31832d,3,credit_card,116.85,sports_leisure,1.0,1.0
99438,fffe18544ffabc95dfada21779c9644f,3,credit_card,64.71,computers_accessories,1.0,1.0


### Merging orders_items/products/payments

In [210]:
orders=orders.dropna()
orders[['order_purchase_timestamp','order_approved_at','order_delivered_customer_date']]= orders[['order_purchase_timestamp','order_approved_at','order_delivered_customer_date']].apply(pd.to_datetime)
orders['purchase_date']=orders.order_purchase_timestamp.dt.to_period('M')
orders['approval_time']= (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.days
orders['delivery_time'] = (orders['order_delivered_customer_date']-orders['order_approved_at']).dt.days
orders.delivery_time =orders.delivery_time.astype(int)
df_orders = orders[['order_id','customer_id','delivery_time','purchase_date']]

In [211]:
df_commande.category.nunique()

71

In [212]:
df_client = pd.merge(df_commande,df_orders,how='left',left_on='order_id',right_on='order_id')

In [213]:
df_client['months']=df_client.purchase_date.dt.month
season_dict = {1: 'Winter',
               2: 'Winter',
               3: 'Spring', 
               4: 'Spring',
               5: 'Spring',
               6: 'Summer',
               7: 'Summer',
               8: 'Summer',
               9: 'Autumn',
               10: 'Autumn',
               11: 'Autumn',
               12: 'Winter'}
df_client['Season'] = df_client['months'].map(season_dict)

In [214]:
df_client.category.nunique()

71

### REVIEW

In [215]:
df_review = order_reviews.groupby('order_id',).agg(rating = ('review_score','mean'),review = ('review_id','count')).reset_index()


### Merging df_client et df_review

In [216]:
df_merge = pd.merge(df_client,df_review,how='left',left_on='order_id',right_on='order_id')
df_merge

,order_id,payment_installment,payment_type,total_pay,category,items_commande,prod_commande,customer_id,delivery_time,purchase_date,months,Season,rating,review
0,00010242fe8c5a6d1ba2dd792cb16214,2,credit_card,72.19,cool_stuff,1.0,1.0,3ce436f183e68e07877b285a838db11a,7.0,2017-09,9,Autumn,5.0,1.0
1,00018f77f2f0320c557190d7a144bdd3,3,credit_card,259.83,pet_shop,1.0,1.0,f6dd3ec061db4e3987629fe6b26e5cce,16.0,2017-04,4,Spring,4.0,1.0
2,000229ec398224ef6ca0657da4fc703e,5,credit_card,216.87,furniture_decor,1.0,1.0,6489ae5e4333f3693df5ad4372dab6d3,7.0,2018-01,1,Winter,5.0,1.0
3,00024acbcdf0a6daa1e931b038114c75,2,credit_card,25.78,perfumery,1.0,1.0,d4eb9395c8c0431ee92fce09860c5a06,6.0,2018-08,8,Summer,4.0,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,3,credit_card,218.04,garden_tools,1.0,1.0,58dbd0b2d70206bf40e62cd34e84d795,25.0,2017-02,2,Winter,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,1,boleto,343.40,housewares,1.0,1.0,b51593916b4b8e0d6f66f2ae24f2673d,15.0,2018-04,4,Spring,5.0,1.0
99436,fffcd46ef2263f404302a634eb57f7eb,1,boleto,386.53,computers_accessories,1.0,1.0,84c5d4fbaf120aae381fad077416eaa0,6.0,2018-07,7,Summer,5.0,1.0
99437,fffce4705a9662cd70adb13d4a31832d,3,credit_card,116.85,sports_leisure,1.0,1.0,29309aa813182aaddc9b259e31b870e6,3.0,2017-10,10,Autumn,5.0,1.0
99438,fffe18544ffabc95dfada21779c9644f,3,credit_card,64.71,computers_accessories,1.0,1.0,b5e6afd5a41800fdf401e0272ca74655,1.0,2017-08,8,Summer,5.0,1.0


In [217]:
df_merge.category.unique()

array(['cool_stuff', 'pet_shop', 'furniture_decor', 'perfumery',
       'garden_tools', 'housewares', 'telephony', 'health_beauty',
       'books_technical', 'fashion_bags_accessories', 'bed_bath_table',
       'sports_leisure', 'consoles_games', 'office_furniture', nan,
       'luggage_accessories', 'food', 'agro_industry_and_commerce',
       'electronics', 'computers_accessories',
       'construction_tools_construction', 'audio', 'baby',
       'construction_tools_lights', 'toys', 'stationery',
       'industry_commerce_and_business', 'watches_gifts', 'auto',
       'home_appliances', 'kitchen_dining_laundry_garden_furniture',
       'air_conditioning', 'home_confort', 'fixed_telephony',
       'small_appliances_home_oven_and_coffee', 'diapers_and_hygiene',
       'signaling_and_security', 'musical_instruments',
       'small_appliances', 'costruction_tools_garden', 'art',
       'home_construction', 'books_general_interest', 'party_supplies',
       'construction_tools_safety', 'c

In [218]:
df_merge2 = pd.merge(df_merge,customers,how='left',left_on='customer_id',right_on='customer_id')
df_merge2.drop(['customer_id','customer_zip_code_prefix'],axis=1,inplace=True)
df_merge2.dropna()
df_merge2.category.fillna('None',inplace=True)
sum_list = ['total_pay','items_commande','prod_commande','review']
mod_list = ['customer_city','customer_state','category','payment_type']#,'payment_type','category']
max_list = ['purchase_date']
last_list = ['Season','months']
mean_list = ['rating','delivery_time']
d1 = dict.fromkeys(sum_list, 'sum')
d2 = dict.fromkeys(mod_list, get_mode)
d3 = dict.fromkeys(max_list,'max')
d4 = dict.fromkeys(last_list,'last')
d5 = dict.fromkeys(mean_list,'mean')
d = {**d1, **d2,**d3,**d4,**d5}

,order_id,payment_installment,payment_type,total_pay,category,items_commande,prod_commande,delivery_time,purchase_date,months,Season,rating,review,customer_unique_id,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,2,credit_card,72.19,cool_stuff,1.0,1.0,7.0,2017-09,9,Autumn,5.0,1.0,871766c5855e863f6eccc05f988b23cb,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,3,credit_card,259.83,pet_shop,1.0,1.0,16.0,2017-04,4,Spring,4.0,1.0,eb28e67c4c0b83846050ddfb8a35d051,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,5,credit_card,216.87,furniture_decor,1.0,1.0,7.0,2018-01,1,Winter,5.0,1.0,3818d81c6709e39d06b2738a8d3a2474,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,2,credit_card,25.78,perfumery,1.0,1.0,6.0,2018-08,8,Summer,4.0,1.0,af861d436cfc08b2c2ddefd0ba074622,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,3,credit_card,218.04,garden_tools,1.0,1.0,25.0,2017-02,2,Winter,5.0,1.0,64b576fb70d441e8f1b2d7d446e483c5,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,1,boleto,343.40,housewares,1.0,1.0,15.0,2018-04,4,Spring,5.0,1.0,0c9aeda10a71f369396d0c04dce13a64,sao luis,MA
99436,fffcd46ef2263f404302a634eb57f7eb,1,boleto,386.53,computers_accessories,1.0,1.0,6.0,2018-07,7,Summer,5.0,1.0,0da9fe112eae0c74d3ba1fe16de0988b,curitiba,PR
99437,fffce4705a9662cd70adb13d4a31832d,3,credit_card,116.85,sports_leisure,1.0,1.0,3.0,2017-10,10,Autumn,5.0,1.0,cd79b407828f02fdbba457111c38e4c4,sao paulo,SP
99438,fffe18544ffabc95dfada21779c9644f,3,credit_card,64.71,computers_accessories,1.0,1.0,1.0,2017-08,8,Summer,5.0,1.0,eb803377c9315b564bdedad672039306,vinhedo,SP


In [284]:
df_last = df_merge2.groupby('customer_unique_id').agg(d).reset_index()
rfm = pd.read_csv('data.csv')
rfm.drop(['zipcode','city','state','total','last_purchase','no_purchase_period'],axis=1,inplace=True)

In [369]:
df = pd.merge(df_last,rfm,how='left',left_on='customer_unique_id',right_on = 'id')
df.drop(['id','purchase_date','customer_unique_id'],axis=1,inplace=True)
df.dropna(how = 'any',inplace=True)

In [371]:
df[df.category =="None"].count()

total_pay         1330
items_commande    1330
prod_commande     1330
review            1330
customer_city     1330
customer_state    1330
category          1330
payment_type      1330
Season            1330
months            1330
rating            1330
delivery_time     1330
freq              1330
recense           1330
montant           1330
dtype: int64

In [372]:
transformer = {'Meubles et décoration': ['bed_bath_table','furniture_decor','office_furniture','home_comfort_2','home_confort', 'housewares', 'furniture_living_room','kitchen_dining_laundry_garden_furniture', 'furniture_bedroom','furniture_mattress_and_upholstery'],
                'Électronique et informatique': ['audio','fixed_telephony','telephony', 'computers_accessories', 'electronics', 'consoles_games', 'computers', 'signaling_and_security','tablets_printing_image', 'cds_dvds_musicals', 'dvds_blu_ray', 'music'],
                'Santé et beauté': ['health_beauty', 'perfumery'],
                'Mode et accessoires': ['fashion_bags_accessories', 'fashion_underwear_beach', 'fashion_shoes', 'fashion_male_clothing', 'fashion_sport', 'luggage_accessories','fashio_female_clothing', 'fashion_childrens_clothes'],
                'Petits appareils ménagers': ['home_appliances','home_appliances_2','small_appliances', 'small_appliances_home_oven_and_coffee', 'air_conditioning'],
                'Outils et matériaux de construction': ['construction_tools_construction','construction_tools_safety','construction_tools_construction,' 'construction_tools_safety', 'construction_tools_lights', 'costruction_tools_garden', 'costruction_tools_tools','home_construction'],
                'Loisirs et sports':['sports_leisure', 'garden_tools', 'musical_instruments', 'cool_stuff', 'toys', 'books_general_interest', 'books_technical', 'books_imported', 'cine_photo', 'art', 'arts_and_craftmanship'],
                'Autres': ['stationery',  'watches_gifts', 'auto', 'christmas_supplies', 'drinks','baby', 'pet_shop', 'market_place', 'flowers', 'industry_commerce_and_business', 'food_drink', 'food', 'agro_industry_and_commerce', 'party_supplies', 'diapers_and_hygiene', 'security_and_services','la_cuisine'],
                'Inconnu' : ['None']
}        
keys =transformer.keys()
for key in keys:
  df.category = df.category.replace(to_replace=transformer[key], value=key)


In [376]:
df

,total_pay,items_commande,prod_commande,review,customer_city,customer_state,category,payment_type,Season,months,rating,delivery_time,freq,recense,montant
0,141.90,1.0,1.0,1.0,cajamar,SP,Meubles et décoration,credit_card,Spring,5,5.0,6.0,3,6.0,3
1,27.19,1.0,1.0,1.0,osasco,SP,Santé et beauté,credit_card,Spring,5,4.0,2.0,3,6.0,1
2,86.22,1.0,1.0,1.0,sao jose,SC,Autres,credit_card,Spring,3,3.0,25.0,0,0.0,2
3,43.62,1.0,1.0,1.0,belem,PA,Électronique et informatique,credit_card,Autumn,10,4.0,20.0,0,0.0,1
4,196.89,1.0,1.0,1.0,sorocaba,SP,Électronique et informatique,credit_card,Autumn,11,5.0,13.0,0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93336,2067.42,2.0,2.0,1.0,sanharo,PE,Santé et beauté,credit_card,Summer,6,5.0,27.0,0,0.0,10
93337,84.58,1.0,1.0,1.0,feira de santana,BA,Autres,credit_card,Winter,12,4.0,30.0,0,0.0,2
93338,112.46,1.0,1.0,1.0,sinop,MT,Autres,credit_card,Winter,2,5.0,14.0,0,0.0,2
93339,133.69,1.0,1.0,1.0,bom jesus do norte,ES,Autres,credit_card,Spring,5,5.0,11.0,3,6.0,3


In [375]:
df.to_csv('merged.csv',index=False)